# Wrapping the Fine-Tuned Model into an Agentic pipeline (LangChain)

## Dependencies

### Modules 

In [ ]:
%pip install langchain

In [ ]:
%pip install langchain_community

In [ ]:
%pip install langchain-huggingface

In [ ]:
%pip install -U langchain-huggingface transformers

### Imports

In [5]:
import langchain, langchain_core, langchain_community
print(langchain.__version__, langchain_core.__version__, langchain_community.__version__)


0.3.26 0.3.74 0.3.27


In [35]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
from langchain.agents import create_react_agent, AgentExecutor, create_tool_calling_agent
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_huggingface import HuggingFacePipeline 
from langchain_huggingface import ChatHuggingFace
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, AutoModelForCausalLM
import numpy as np
import torch
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory

In [7]:
use_gpu = torch.cuda.is_available()
device = 0 if use_gpu else -1

## Agent

In [8]:
gen_model_path = "Qwen/Qwen2.5-1.5B-Instruct"

In [9]:
gen_tok = AutoTokenizer.from_pretrained(gen_model_path)
gen_model = AutoModelForCausalLM.from_pretrained(gen_model_path)
gen_pipe = pipeline("text-generation",
                    model=gen_model, 
                    tokenizer=gen_tok, 
                    max_new_tokens=256,
                    
                    device=device
                   )

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cuda:0


In [10]:
llm = ChatHuggingFace(llm=HuggingFacePipeline(pipeline=gen_pipe))

In [13]:
classifier_path = "/kaggle/input/classifier-model/bert-spam-ham-classifier-full_dataset/checkpoint-13086"
cls_model = AutoModelForSequenceClassification.from_pretrained(classifier_path)
cls_tokenizer = AutoTokenizer.from_pretrained(classifier_path)

In [14]:
clf = pipeline("text-classification", 
               model=cls_model, 
               tokenizer=cls_tokenizer, 
               device=device
              )

Device set to use cuda:0


In [62]:
clf("hello click here")

[{'label': 'LABEL_1', 'score': 0.6220700144767761}]

In [16]:
@tool("classify_spam_ham")
def classify_spam_ham(text: str) -> str:
    """
    Classify the given message as spam or ham.
    """
    pred = clf(text)[0]["label"]
    return "spam" if pred == "LABEL_1" else "ham"

In [17]:
tools = [classify_spam_ham]

In [87]:
system = """
You are a terminal assistant.

RULES:
- Only call the "classify_spam_ham" tool if the user EXPLICITLY asks to classify,
  OR they use trigger phrases like: "classify", "is this spam", "spam or ham",
  "label this", "check for spam", or the input starts with "Classify this:" or "Classify:".
- If the user sends a greeting or small talk (e.g., "hello", "hi"), DO NOT call any tool.
- When you do call the tool, after it returns, output EXACTLY the tool's result
  in lowercase (`spam` or `ham`), with some extra words saying its a spam or ham
"""

In [88]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    MessagesPlaceholder("chat_history"),      
    ("human", "{input}"),                     
    MessagesPlaceholder("agent_scratchpad"),
])

In [89]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [90]:
memory = ConversationBufferWindowMemory(
    k=20,
    return_messages=True,
    memory_key="chat_history",
    output_key="output"
)

In [91]:
agent_exec = AgentExecutor(agent=agent, 
                           tools=tools,
                           memory=memory,
                           verbose=True,
                           memory_key="chat_history",
                           return_intermediate_steps=True)

In [92]:
res = agent_exec.invoke({"input": "Classify this: You won $10,000! Claim now."})
print(res["output"])



> Entering new AgentExecutor chain...
spam

> Finished chain.
spam


In [93]:
res = agent_exec.invoke({"input": "Who is Michael Jordan?"})
print(res["output"])



> Entering new AgentExecutor chain...
Michael Jordan was an American professional basketball player who had an incredible career spanning multiple decades. He played his entire NBA career with the Chicago Bulls and earned numerous accolades throughout his tenure. Some of his most notable achievements include winning six NBA championships, five MVP awards, and being named the NBA Most Valuable Player three times. Jordan also starred in several films and TV shows, including the movie "Space Jam," which featured him playing against the Los Angeles Lakers' team. His influence on sports and culture continues to be felt today.

> Finished chain.
Michael Jordan was an American professional basketball player who had an incredible career spanning multiple decades. He played his entire NBA career with the Chicago Bulls and earned numerous accolades throughout his tenure. Some of his most notable achievements include winning six NBA championships, five MVP awards, and being named the NBA Most V

In [94]:
res = agent_exec.invoke({"input": "What's my name?"})
print(res["output"])



> Entering new AgentExecutor chain...
I'm sorry, but I don't have access to your personal information such as your name. I can only provide general responses based on publicly available data that might not always be accurate or up-to-date.

> Finished chain.
I'm sorry, but I don't have access to your personal information such as your name. I can only provide general responses based on publicly available data that might not always be accurate or up-to-date.


In [95]:
res = agent_exec.invoke({"input": "my name is Bob"})
print(res["output"])



> Entering new AgentExecutor chain...
Bob

> Finished chain.
Bob


In [96]:
res = agent_exec.invoke({"input": "What's my name?"})
print(res["output"])



> Entering new AgentExecutor chain...
Your name is Bob.

> Finished chain.
Your name is Bob.


In [97]:
res = agent_exec.invoke({"input": "What's my favorite color?"})
print(res["output"])



> Entering new AgentExecutor chain...
I don't have access to your personal preferences or memories, so I cannot determine what your favorite color is.

> Finished chain.
I don't have access to your personal preferences or memories, so I cannot determine what your favorite color is.


In [98]:
res = agent_exec.invoke({"input": "My favorite color is yellow"})
print(res["output"])



> Entering new AgentExecutor chain...
Yellow is one of my favorite colors too!

> Finished chain.
Yellow is one of my favorite colors too!


In [99]:
res = agent_exec.invoke({"input": "What's my favorite color?"})
print(res["output"])



> Entering new AgentExecutor chain...
Yellow is one of my favorite colors too!

> Finished chain.
Yellow is one of my favorite colors too!


In [100]:
res

{'input': "What's my favorite color?",
 'chat_history': [HumanMessage(content='Classify this: You won $10,000! Claim now.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='spam', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Who is Michael Jordan?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Michael Jordan was an American professional basketball player who had an incredible career spanning multiple decades. He played his entire NBA career with the Chicago Bulls and earned numerous accolades throughout his tenure. Some of his most notable achievements include winning six NBA championships, five MVP awards, and being named the NBA Most Valuable Player three times. Jordan also starred in several films and TV shows, including the movie "Space Jam," which featured him playing against the Los Angeles Lakers\' team. His influence on sports and culture continues to be felt today.', additional_kwargs={}, response_metadata={}),
  Hu